<a href="https://colab.research.google.com/github/jpbraz/nanomed-colab-prophet-googlesheets/blob/main/profeta_datasus_googlesheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de Pesquisa NANOMED - UFRN/EMCM
    
  ## Grupo:
    Profa. Dra. Janine Braz (Coodenadora)
    Prof. Dr. Diego Bonfada (Coordenador Adjunto)
    Prof. Dr. Leonardo Nobre (Pesquisador)
    MS.c Brígida Albuquerque (Pesquisadora)
    Prof. Dr. Tiago Oliveira (Pesquisador)
    Profa. Dra. Carla Nascimento (Pesquisadora)
    Aurélio Neto (Discente de Medicina)
    David Nery (Discente de Medicina)
    João Paulo (Discente de Tecnologia da Informação)
##  

Executar cada área de código, clicando no botão "play" que aparece do lado esquerdo de célula de código neste documento, seguindo a ordem abaixo.

Obs.:
1.   Caso seja necessário alterar alguma variável no código, basta clica na região que deseja alterar e editar antes de executar.
2.   Pode-se executar todo o código pressionando Ctrl+F9 ou selecionando no menu do 'googlecolab'.
2.   Na execução de alguns os códigos são emitidos saídas (outputs) abaixo do código, mostrando o resultado de cada execução, quando a possui (para alguns não tem saída).

## 1ª Etapa

Instalação do pacote "Gspread", para manipular planilhas do google drive, usando o comando "pip" do python. Para isso vamos utilizar a *tag* '--upgrade', para garantir a instalação da última versão.
Copie e cole: !pip install --upgrade gspread

👇

In [ ]:
!pip install gspread==3.4.2

## 2ª Etapa

Autenticação para conseguir ler as planilhas do google sheets, que estão no google drive.

👇

In [ ]:
from google.colab import auth
auth.authenticate_user()



#### Para isso:
1) Clica no link gerado.

2) Realiza a autenticação em sua conta google, autorizando o google cloud SDK a acessar a sua conta.

3) Copia o código gerado e insere no espaço de verificação acima.


Informandos as credenciais do usuário autenticado para o google sheet, conseguir ler as planilhas, que estão no google drive.

👇

In [ ]:
import gspread
from google.auth import default
creds, _ = default()

gs = gspread.authorize(creds)

## 3ª Etapa

Seleciona a planilha do google sheets que você que quer trabalhar, passando o nome da planilha para o método "gs.open".

Selecione o nome da planilha do seu Google Drive, na caixa de seleção que aparecerá abaixo do código

👇

In [ ]:
import ipywidgets as widgets

# recupera uma lista de arquivos do seu google drive
minhas_planilhas = gs.list_spreadsheet_files()

# cria uma lista com tuplas (chave, valor) dos nomes das planilhas recuperadas acima
nomes_planilhas = []
for cada_planilha in minhas_planilhas:
  nomes_planilhas.append((cada_planilha['name'], cada_planilha['name']))

# mostra uma caixa de seleção para selecionar o arquivo que deseja trabalhar
planilha_selecionada = widgets.Dropdown(options=nomes_planilhas, value='pesquisa_caico')
planilha_selecionada


A planilha será armazenanda na variável "planilha"

👇

In [ ]:
planilha = gs.open(planilha_selecionada.value)
print(planilha.title)

In [ ]:
# recupera uma lista de abas do arquivo selecionado no passo anterior
sheets = planilha.worksheets()

# cria uma lista com os nomes das abas do arquivo recuperado no passso anterior
nomes_abas = []
for sheet in sheets:
  nomes_abas.append((sheet.title))

# mostra uma caixa de seleção para selecionar o arquivo que deseja trabalhar
aba_selecionada = widgets.Dropdown(options=nomes_abas)

## **Importante**: Selecionar aqui a **planilha**
Execute a célula abaixo e seleciona a aba, na caixa de seleção que aparecerá abaixo

👇

In [ ]:
aba_selecionada

Abaixo é realizada a seleção da planilha pelo o número, mostrando o nome de "pagina" (aba). Ex.: Para a 1ª planilha do arquivo, usa planilha.get_worksheet(0); Para a 2ª, planilha.get_worksheet(1), etc

👇

In [ ]:
pagina = planilha.get_worksheet(aba_selecionada.index)

print("Planilha selecionada: ", pagina.title)

Verificar o cabeçalho da planilha

In [ ]:
pagina.row_values(1)

## 4ª Etapa
Carregar a planilha em um Dataframe do Pandas.
Dataframes pandas são objetos de duas dimensões (tabela), no qual cada coluna pode possuir um tipo primário diferente. São utilizados para armazenar bases de dados.

### Para isso, vamos:

1) Importa o pacote pandas

👇

In [ ]:
import pandas as pd

2) Pega todos os registro e armazena na memória como um Dataframe

👇

In [ ]:
df = pd.DataFrame(pagina.get_all_records())

3) Mostra as primeiras linhas do dataframe

In [ ]:
df.head()

4) Mostra as últimas linhas do dataframe

In [ ]:
df.tail()

Pode-se utilizar o método query para filtrar um valor específico de uma coluna.
Ex.: df.query('Ano'==2020)

```
df.query('Ano in [2018,2019]')`
```
```
# para verificar o formato da células
info = df.query('Ano==2018-12-31').info()
```

In [ ]:
# numero de linhas
df.shape[0]

## 5ª Etapa
Fazer a predição com o [Prophet by Facebook](https://facebook.github.io/prophet/docs/quick_start.html)
- Profeta é um procedimento para prever dados de série temporal com base em um modelo aditivo em que tendências não lineares são ajustadas com sazonalidade anual, semanal e diária, além de efeitos de feriados. Funciona melhor com séries temporais que têm fortes efeitos sazonais e várias temporadas de dados históricos. O Prophet é robusto para dados ausentes e mudanças na tendência, e normalmente lida bem com outliers. ([Informações copiadas de sua documentação](https://facebook.github.io/prophet/))



1.   Instalar
2.   Importar
3.   Predição


👇👇...👇

In [ ]:
!pip install pystan==2.19.1.1

!pip install prophet


In [ ]:
from prophet import Prophet

In [ ]:
head = pagina.row_values(1)

# mostra uma caixa de seleção para selecionar a coluna com a data
coluna_data = widgets.Dropdown(options=head.copy(), value=head[0], description='Seleciona a coluna com a Data (Série Temporal): ')
coluna_data

In [ ]:
coluna_data.index
head.remove(coluna_data.value)
lista_dados = []
lista_dados = head

# mostra uma caixa de seleção para selecionar a coluna com os dados a serem analisados
coluna_valores = widgets.Dropdown(options=lista_dados, value = lista_dados[0], description='Selecione a coluna com os dados a serem analisados: ')
coluna_valores


In [ ]:
df.rename(columns={coluna_data.value: 'ds', coluna_valores.value: 'y'}, inplace = True)
# df.info()

In [ ]:
m = Prophet()
m.fit(df)

In [ ]:
# Periodo de previsão futura
periodo_previsao = 4
frequencia = 'Y' # Y: Anual / M: Mensal / D: Diária / ms: milisegundos / h: horas / mim: minutos
future = m.make_future_dataframe(periods=periodo_previsao, freq=frequencia, include_history=True)
# future.tail(21)

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]


In [ ]:
fig1 = m.plot(forecast, xlabel='Ano', ylabel='Casos')


In [ ]:
fig2 = m.plot_components(forecast)





#DIAGNOSTICO - Cross Validation

In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics

numero_dias_no_ano = 365
quantidade_de_anos_analisados= df.shape[0]

quantidade_para_treino = quantidade_de_anos_analisados *0.50
quantidade_para_horizonte = quantidade_de_anos_analisados * 0.25

# o tamanho do período de treinamento inicial (em dias)
periodo_de_treino = quantidade_para_treino * numero_dias_no_ano

# periodo de cada corte (em dias)
periodo_de_corte = numero_dias_no_ano

# horizonte de previsão (em dias)
horizonte = quantidade_para_horizonte * numero_dias_no_ano

# dataframe com os valores reais y e os valores de previsão fora da amostra yhat, em cada data de previsão simulada e para cada data de corte.
df_cv = cross_validation(m, initial=f'{periodo_de_treino} days', period=f'{periodo_de_corte} days', horizon=f'{horizonte} days')

# métricas de desempenho - MSE, RMSE, MAE, MAPE e MDAPE
metrics = performance_metrics(df_cv)
metrics

In [ ]:
# Python
from prophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='mse')


.

.

.

## Modelo para uso do pyMannKendall


# 👇

In [ ]:
!pip install pymannkendall

Análise de Tendência dos valores após análise de predição pelo profeta

👇

In [ ]:
import pymannkendall as mk

result = mk.original_test(forecast[['yhat']])
print(result)

## Útima etapa
Pegar os valores e salvar em uma página da planilha
#✋ ***Atenção! Só executar esta etapa, caso queira gravar na planilha.***



In [ ]:
import datetime
# gerando um novo nome baseado no tempo
x = datetime.datetime.now()
nome_nova_pagina = "analise "+ x.strftime("%d/%m/%Y-%X")

# criar uma página na planilha com x colunas e y linhas

pagina_com_estimativas = planilha.add_worksheet(nome_nova_pagina, rows=forecast.shape[0], cols=forecast.shape[1])

In [ ]:
forecast['ds'] = forecast['ds'].astype(str)
# transferir os dados do dataframe para a planilha
pagina_com_estimativas.update([forecast.columns.values.tolist()] + forecast.values.tolist())